# this script reads in tsv files with txt and copmutes their opinion score

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import html.parser
h = html.parser.HTMLParser()

## function to add median score of comment

In [ ]:
def add_score_col(df):
    medians = []
    for g in  df['body']:
        if str(g)!='nan':
            in_body = h.unescape(str(g))
            in_sentences = in_body.replace("e.g.","eg").replace("?",".").split('.')
            # clean short things ("..." etc.)
            in_sentences = np.array([ins for ins in in_sentences if len(ins)>20])
            in_sentences = np.array([ins for ins in in_sentences if ins[0]!=">"])
            scores = []
            in_sentences = [str(in_sentences[k:k+1][0])+str(in_sentences[k+1:k+2])
                            for k in range(len(in_sentences))]

            for ins_words in in_sentences:
                ins = tokenizer.texts_to_sequences([ins_words])
                ins = np.array(sequence.pad_sequences(ins, maxlen=maxlen))
                y = model.predict(ins)[0][0]
                scores.append(y)
        medians.append(np.median(scores))
        df['median_scores'] = medians

## include the model

In [43]:
from keras.preprocessing import sequence
from keras.models import load_model
import pickle

model = load_model('data_files/abortion/bilstm.h5')
# loading
with open('data_files/abortion/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
maxlen = 100

/Users/yuki/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## do the stuff

In [71]:
for k in range(1,11):
    df = pd.read_csv('target_conversations/output/cmv1_chain{}.tsv'.format(k))
    add_score_col(df)
    df.to_csv('target_conversations/output/cmv1_chain{}_score.tsv'.format(k)

/Users/yuki/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  after removing the cwd from sys.path.
/Users/yuki/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/yuki/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
